In [ ]:
# dependencies
import pandas as pd
from sqlalchemy import create_engine

# Taking out of CSV

In [ ]:
# importing csv's
scotch_file = 'Resources/scotch_score.csv'
scotch_region_df = pd.read_csv(scotch_file)
scotch_region_df.head()

In [ ]:
scotch_region_df['NAME'].nunique()

In [ ]:
scotch_region_df.dtypes

In [ ]:
whisky_file = 'Resources/whisky.csv'
whisky_df = pd.read_csv(whisky_file)
whisky_df.head()

In [ ]:
whisky_df.Distillery.nunique()

In [ ]:
# whisky_df.dtypes

# Transforming Data

In [ ]:
# keeping only name, region and district columns
scotch_df = scotch_region_df[['NAME','REGION','DISTRICT']].copy()
scotch_df.head()

In [ ]:
# changing names (i like those names, but I want them lowercase)
scotch_df = scotch_df.rename(columns={'NAME': 'distillery', 'REGION': 'region', 'DISTRICT': 'district'})
scotch_df.head()

In [ ]:
# there are duplicates because they were going by bottle and we want to go by distillery only
scotch_df.drop_duplicates(inplace=True)
scotch_df

In [ ]:
# now there are some NaN values in there
scotch_df.dropna(inplace=True)
scotch_df.count()

In [ ]:
whisky_df.columns

In [ ]:
# on to whisky_df
# making df for flavors
flavor_df = whisky_df[['Distillery', 'Body', 'Sweetness', 'Smoky', 'Medicinal',
       'Tobacco', 'Honey', 'Spicy', 'Winey', 'Nutty', 'Malty', 'Fruity',
       'Floral']].copy()
flavor_df.head()

In [ ]:
# renaming columns lowercase (again, these are pretty good column names already)
flavor_df = flavor_df.rename(columns={'Distillery': 'distillery', 'Body': 'body', 'Sweetness': 'sweetness',
                                     'Smoky': 'smoky', 'Medicinal': 'medicinal', 'Tobacco': 'tobacco', 
                                     'Honey': 'honey', 'Spicy': 'spicy', 'Winey': 'winey', 'Nutty': 'nutty',
                                     'Malty': 'malty', 'Fruity': 'fruity', 'Floral': 'floral'})
flavor_df.head()

In [ ]:
# lets make a lat/long df
whisky_df.head()

In [ ]:
postal_df = whisky_df[['Distillery', 'Postcode']].copy()
postal_df.head()

In [ ]:
# rename columns
postal_df = postal_df.rename(columns={'Distillery': 'distillery', 'Postcode': 'postcode'})
postal_df.head()

In [ ]:
scotch_df.region.unique()

In [ ]:
# I want to change those three to Highlands, Lowlands, and Islay
scotch_df = scotch_df.replace(to_replace={'HIGH': 'Highlands', 'ISLAY': 'Islay', 'LOW': 'Lowlands'})
scotch_df.head()

In [ ]:
scotch_df.loc[scotch_df.region == 'Highlands'].district.unique()

In [ ]:
scotch_df.loc[scotch_df.region == 'Lowlands'].district.unique()
# they reused districts like east and west between different regions.
# and i kind of like it that way so I am going to leav it

In [ ]:
scotch_df.loc[scotch_df.region == 'Islay'].district.unique()

In [ ]:
scotch_df.district.unique()

In [ ]:
# replacing names of scotch districts
scotch_df = scotch_df.replace(to_replace={'MIDLAND': 'Midland', 'SPEY': 'Speyside', 'SOUTH': 'South',
                                          'WEST': 'West', 'NORTH': 'North', 'BORDERS': 'Borders', 'LOCH': 'Loch',
                                           'EAST': 'East', 'CAMPBEL': 'Campbeltown', 'ORKNEY': 'Orkney',
                                          'JURA': 'Jura', 'NORTHWEST': 'Northwest', 'CENTRAL': 'Central',
                                          'SKYE': 'Skye', 'MULL': 'Mull'})
scotch_df.district.unique()

In [ ]:
# drop \t on postcode in postal_df
for x in range(len(postal_df.postcode)):
    post = postal_df.postcode[x]
    if '\t' in post:
        post = post.split('\t')[1]
        postal_df.postcode[x] = post
        

In [ ]:
postal_df.postcode.head()

In [ ]:
postal_df.count()

In [ ]:
#look at three dfs
flavor_df.dtypes

In [ ]:
scotch_df.dtypes

In [ ]:
postal_df.dtypes

# Putting into Database

In [ ]:
from config import user, password, host, port

In [ ]:
# connecting to scotch_db database
db = 'scotch_db'
rds_connection_string = f"{user}:{password}@{host}:{port}/{db}"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [ ]:
# looking at tables in scotch_db
engine.table_names()

In [ ]:
# loading scotch df to scotch region table
scotch_df.to_sql(name='scotch_region', con=engine, if_exists='replace', index=False)

In [ ]:
# loading flavor df to scotch_flavor table
flavor_df.to_sql(name='scotch_flavor', con=engine, if_exists='replace', index=False)

In [ ]:
# loading scotch df to scotch region table
postal_df.to_sql(name='scotch_location', con=engine, if_exists='replace', index=False)

We chose to do if_exists='replace' rather than append because we are putting in the same data each time and if we ran the code again we didn't want to create duplicates of the data.

In [ ]:
# just looking at distillery names
flavor_df.distillery.unique()